<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_3C_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises
https://pgexercises.com/ <br>
https://pgexercises.com/gettingstarted.html

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-08-13 05:27:46.786239+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install & Configure Postgres

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!sudo -u postgres psql -V
!psql -V
!service postgresql start



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


psql (PostgreSQL) 14.12 (Ubuntu 14.12-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
#!pip install psycopg2
import psycopg2
import pandas as pd

#Create Schema, Tables and Insert Data

In [4]:
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"

ALTER ROLE


In [5]:
# Download and execute SQL file to create tables, load data
#
#!wget https://pgexercises.com/dbfiles/clubdata.sql
!wget -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/clubdata.sql
#Run the batch file to create tables, load data
!psql -h localhost -U postgres -f clubdata.sql -d postgres -x -q


Password for user postgres: 


In [6]:
# Create file to store the password. Else will be prompted each time
#!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!echo "localhost:5432:exercises:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:exercises:postgres:pass


In [7]:
#Note new database called exercises
!psql -h localhost -p 5432 -Upostgres -c "\l"

Password for user postgres: 
                                  List of databases
   Name    |  Owner   | Encoding |   Collate   |    Ctype    |   Access privileges   
-----------+----------+----------+-------------+-------------+-----------------------
 exercises | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 postgres  | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 template0 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
 template1 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
(4 rows)



In [8]:
#Note three tables under schema cd
!psql -h localhost -p 5432 -Upostgres -d exercises -c "\dt+ cd.*"

                                      List of relations
 Schema |    Name    | Type  |  Owner   | Persistence | Access method |  Size  | Description 
--------+------------+-------+----------+-------------+---------------+--------+-------------
 cd     | bookings   | table | postgres | permanent   | heap          | 264 kB | 
 cd     | facilities | table | postgres | permanent   | heap          | 16 kB  | 
 cd     | members    | table | postgres | permanent   | heap          | 16 kB  | 
(3 rows)



In [9]:
#Note column / datatypes of all tables under schema cd
!psql -h localhost -p 5432 -Upostgres -d exercises -c "\d cd.*"

                           Table "cd.bookings"
  Column   |            Type             | Collation | Nullable | Default 
-----------+-----------------------------+-----------+----------+---------
 bookid    | integer                     |           | not null | 
 facid     | integer                     |           | not null | 
 memid     | integer                     |           | not null | 
 starttime | timestamp without time zone |           | not null | 
 slots     | integer                     |           | not null | 
Indexes:
    "bookings_pk" PRIMARY KEY, btree (bookid)
    "bookings.facid_memid" btree (facid, memid)
    "bookings.facid_starttime" btree (facid, starttime)
    "bookings.memid_facid" btree (memid, facid)
    "bookings.memid_starttime" btree (memid, starttime)
    "bookings.starttime" btree (starttime)
Foreign-key constraints:
    "fk_bookings_facid" FOREIGN KEY (facid) REFERENCES cd.facilities(facid)
    "fk_bookings_memid" FOREIGN KEY (memid) REFERENCES cd.mem

In [11]:
#Note column / datatypes of all tables under schema cd
!psql -h localhost -p 5432 -Upostgres -d exercises -c "\d cd.members"

                              Table "cd.members"
    Column     |            Type             | Collation | Nullable | Default 
---------------+-----------------------------+-----------+----------+---------
 memid         | integer                     |           | not null | 
 surname       | character varying(200)      |           | not null | 
 firstname     | character varying(200)      |           | not null | 
 address       | character varying(300)      |           | not null | 
 zipcode       | integer                     |           | not null | 
 telephone     | character varying(20)       |           | not null | 
 recommendedby | integer                     |           |          | 
 joindate      | timestamp without time zone |           | not null | 
Indexes:
    "members_pk" PRIMARY KEY, btree (memid)
    "members.joindate" btree (joindate)
    "members.recommendedby" btree (recommendedby)
Foreign-key constraints:
    "fk_members_recommendedby" FOREIGN KEY (recommendedby

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [12]:
# Local Colab server

conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.12 (Ubuntu 14.12-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [13]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

In [14]:
def pyClient(s):
    if s.split(' ', 1)[0].lower() == 'select':      # If first word = select
        df = runSelect(s)                           # then use runSelect()
        return df                                   # return the pandas dataframe obtained from runSelect()
    else:                                           # else
        runCmd(s)                                   # use runCmd()

##Check Data

In [15]:
#runSelect("select * from cd.members limit 3")
pyClient("select * from cd.members limit 3")

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,None,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,None,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,None,2012-07-02 12:08:23


In [16]:
#runSelect("select * from cd.facilities limit 3")
pyClient("select * from cd.facilities limit 3")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50


In [17]:
#runSelect("select * from cd.bookings limit 3")
pyClient("select * from cd.bookings limit 3")

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


In [18]:
# Testing pyClient for non-Select SQL

#pyClient('create table test(col1 char(10))')
#pyClient("insert into test (col1) values('bbb')")
#pyClient('select * from test')

#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [19]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2024-08-13 05:38:57.628328+05:30
